In [1]:
# import pytorch
import torch

# define a tensor
torch.FloatTensor([2])
torch.cuda.current_device()
torch.cuda.device(0)
 torch.cuda.device_count()
    torch.cuda.get_device_name(0)

tensor([2.])

In [2]:
from torch import nn
from torch.nn import functional as F
import torch
from torchvision import models
import torchvision


def conv3x3(in_, out):
    return nn.Conv2d(in_, out, 3, padding=1)


class ConvRelu(nn.Module):
    def __init__(self, in_, out):
        super().__init__()
        self.conv = conv3x3(in_, out)
        self.activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.activation(x)
        return x


class DecoderBlock(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels):
        super().__init__()

        self.block = nn.Sequential(
            ConvRelu(in_channels, middle_channels),
            nn.ConvTranspose2d(middle_channels, out_channels, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.block(x)


class DecoderBlockV2(nn.Module):
    def __init__(self, in_channels, middle_channels, out_channels, is_deconv=True):
        super(DecoderBlockV2, self).__init__()
        self.in_channels = in_channels

        if is_deconv:
            """
                Paramaters for Deconvolution were chosen to avoid artifacts, following
                link https://distill.pub/2016/deconv-checkerboard/
            """

            self.block = nn.Sequential(
                ConvRelu(in_channels, middle_channels),
                nn.ConvTranspose2d(middle_channels, out_channels, kernel_size=4, stride=2,
                                   padding=1),
                nn.ReLU(inplace=True)
            )
        else:
            self.block = nn.Sequential(
                nn.Upsample(scale_factor=2, mode='bilinear'),
                ConvRelu(in_channels, middle_channels),
                ConvRelu(middle_channels, out_channels),
            )

    def forward(self, x):
        return self.block(x)

In [3]:
class UNet16(nn.Module):
    def __init__(self, num_classes=1, num_filters=32, pretrained=False, is_deconv=False):
        """
        :param num_classes:
        :param num_filters:
        :param pretrained:
            False - no pre-trained network used
            True - encoder pre-trained with VGG16
        :is_deconv:
            False: bilinear interpolation is used in decoder
            True: deconvolution is used in decoder
        """
        super().__init__()
        self.num_classes = num_classes

        self.pool = nn.MaxPool2d(2, 2)

        self.encoder = torchvision.models.vgg16(pretrained=pretrained).features

        self.relu = nn.ReLU(inplace=True)

        self.conv1 = nn.Sequential(self.encoder[0],
                                   self.relu)

        self.conv2 = nn.Sequential(self.encoder[5],
                                   self.relu)

        self.conv3 = nn.Sequential(self.encoder[10],
                                   self.relu)

        self.conv4 = nn.Sequential(self.encoder[17],
                                   self.relu)

        self.conv5 = nn.Sequential(self.encoder[24],
                                   self.relu)

        self.center = DecoderBlockV2(512, num_filters * 8 * 2, num_filters * 8, is_deconv)

        self.dec5 = DecoderBlockV2(512 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv)
        self.dec4 = DecoderBlockV2(512 + num_filters * 8, num_filters * 8 * 2, num_filters * 8, is_deconv)
        self.dec3 = DecoderBlockV2(256 + num_filters * 8, num_filters * 4 * 2, num_filters * 2, is_deconv)
        self.dec2 = DecoderBlockV2(128 + num_filters * 2, num_filters * 2 * 2, num_filters, is_deconv)
        self.dec1 = ConvRelu(64 + num_filters, num_filters)
        self.final = nn.Conv2d(num_filters, num_classes, kernel_size=1)

    def forward(self, x):
        conv1 = self.conv1(x)
        conv2 = self.conv2(self.pool(conv1))
        conv3 = self.conv3(self.pool(conv2))
        conv4 = self.conv4(self.pool(conv3))
        #conv5 = self.conv5(self.pool(conv4))

        center = self.center(self.pool(conv4))

        dec4 = self.dec5(torch.cat([center, conv4], 1))

        #dec4 = self.dec4(torch.cat([dec5, conv4], 1))
        dec3 = self.dec3(torch.cat([dec4, conv3], 1))
        dec2 = self.dec2(torch.cat([dec3, conv2], 1))
        dec1 = self.dec1(torch.cat([dec2, conv1], 1))

        if self.num_classes > 1:
            x_out = F.log_softmax(self.final(dec1), dim=1)
        else:
            x_out = self.final(dec1)

        return x_out

In [6]:
# model = UNet16(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
model = nn.Conv2d(3,1,3, padding=1)
#test = nn.ConvTranspose2d(3, 6, kernel_size=3, stride=2, padding=1, output_padding=1)
input = torch.rand(1, 3, 416, 416)
print(input.size())

torch.Size([1, 3, 416, 416])


In [7]:
output = model(input)
print(output.size())


torch.Size([1, 1, 416, 416])


In [12]:
import plotly
from plotly.offline import iplot, init_notebook_mode
#import plotly.plotly as py
import plotly.graph_objs as go
import plotly.io as pio

import os

#plotly.__version__

init_notebook_mode(connected=True)

plotly.io.orca.config.executable = '/home/royce/orca/orca-1.2.1-x86_64.AppImage'
plotly.io.orca.config.save() 

class potly(object):
    def __init__(self, pot=0):
        self.pot = 0
    

    def save_plot(self, train_y, val_y, output_dir, filename):
#         if(len(train_y) != len(val_y)):
#             print("Train not same length as validation")
        
        # get x axis
        train_x = [i for i in range(len(train_y))]
        val_x = [i for i in range(len(val_y))]
        
        # create plots
        fig1 = go.Scatter(x=train_x, y=train_y, name='training')
        fig2 = go.Scatter(x=val_x, y=val_y, name='validation')
        
        # needed for data input
        data = [fig1, fig2]
        
        # create layout: titles
        layout = go.Layout(
            title='Final Epochs',
            xaxis=dict(
                title='Epoch #',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            ),
            yaxis=dict(
                title='Loss',
                titlefont=dict(
                    family='Courier New, monospace',
                    size=18,
                    color='#7f7f7f'
                )
            )
        )
        
        # load data and layout
        fig = go.Figure(data=data, layout=layout)
        
        # double check to make sure if directory exists
        os.makedirs(output_dir, exist_ok=True)
        
        pio.write_image(fig, output_dir + filename + '.png')


In [1]:
import numpy
#import potly
from datetime import datetime
import json

with open('./data/logs/epoch - 2019-04-23-3.log', 'r') as data_file:
    json_data_1 = data_file.read()

with open('./data/logs/epoch - 2019-04-24-5.log', 'r') as data_file:
    json_data_2 = data_file.read()
    
with open('./data/logs/epoch - 2019-04-25-6.log', 'r') as data_file:
    json_data_3 = data_file.read()
    
with open('./data/logs/epoch - 2019-04-25-7.log', 'r') as data_file:
    json_data_4 = data_file.read()
    
with open('./data/logs/epoch - 2019-04-26-8.log', 'r') as data_file:
    json_data_5 = data_file.read()

data1 = json.loads(json_data_1)
data2 = json.loads(json_data_2)
data3 = json.loads(json_data_3)
data4 = json.loads(json_data_4)
data5 = json.loads(json_data_5)

#pot = potly()
#output_dir = './data/outputs/plots/'
#filename = str(datetime.now().date()) + "-1"
#N = 100
#x = val[0][0]
#y = val[1][0]

#pot.save_plot(x,y,output_dir, filename)
print(data1)
print(data2)
print(data3)
print(data4)
print(data5)

[[[0.6054948834180832, 0.5342046623080969, 0.46604244205355644]], [[0.005458398520946503, 0.005060129016637802, 0.004333792626857758]]]
[[[0.38173164850473407]], [[0.003895449012517929]]]
[[[0.3695145677104592, 0.36339623710513114, 0.3448032429926097]], [[0.00371395069360733, 0.004109779700636864, 0.0034569417238235474]]]
[[[0.3426332968175411, 0.34493934413790706, 0.32968188293278217, 0.321596234023571]], [[0.0036532875150442124, 0.00336752200871706, 0.0032557889595627785, 0.0035381605625152586]]]
[[[0.31752758847177026, 0.32328712229430673, 0.31472743723541496]], [[0.0029422275945544243, 0.0033178173527121544, 0.0031131386756896974]]]


In [7]:
x = []
y = []
for data in [data1, data2, data3, data4, data5]:
    for i in range(len(data[0][0])):
        x.append(data[0][0][i])
        y.append(data[1][0][i])
print(x)
print(y)

[0.6054948834180832, 0.5342046623080969, 0.46604244205355644, 0.38173164850473407, 0.3695145677104592, 0.36339623710513114, 0.3448032429926097, 0.3426332968175411, 0.34493934413790706, 0.32968188293278217, 0.321596234023571, 0.31752758847177026, 0.32328712229430673, 0.31472743723541496]
[0.5458398520946502, 0.5060129016637802, 0.4333792626857758, 0.3895449012517929, 0.371395069360733, 0.41097797006368636, 0.3456941723823547, 0.36532875150442123, 0.336752200871706, 0.32557889595627787, 0.35381605625152585, 0.2942227594554424, 0.33178173527121546, 0.3113138675689697]


In [13]:
pot = potly()
output_dir = './data/outputs/plots/'
filename = "final run"
pot.save_plot(x,y,output_dir, filename)

In [7]:
import os
from random import randint
import dataGenerator
from datetime import datetime
from PIL import Image
import torch

dat = dataGenerator.dataGenerator(416)
random_test_filenames = [randint(0,100000) for _ in range(5)]
outputs_dir = "./data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)

photo_dir = './data/train/photos/'
tensor_dir = './data/train/tensors/'

In [8]:
photo_paths = [photo_dir + str(random_test_filenames[i]) + '.png' for i in range(5)]
tensor_paths = [tensor_dir + str(random_test_filenames[i]) + '.pt' for i in range(5)]
threshold = 1

for i in range(5):
    img = Image.open(photo_paths[i])
    tensor = torch.load(tensor_paths[i])
    #img = dat.showDefaultTensor(img, tensor, threshold)
    img = dat.showMaskOnImage(img, tensor, threshold)
    img.save(outputs_dir  + "/MoI_Test 3 - " + str(random_test_filenames[i]) + ".png","PNG")

In [1]:
import torchvision
torchvision.models.vgg16(pretrained=True).features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(

In [ ]:
dat = dataGenerator.dataGenerator(416)
j = 0
outputs_dir = "data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)
threshold = 0.05
batch = 3

for locations in paths:
    img = Image.open(locations)
    img = dat.showMaskOnImage(img,outputs[j], threshold)
    img.save(outputs_dir  + "/batch_" + str(batch) + "-" + str(j) + ".png","PNG")
    #img.show()
    j += 1

#Cells
outputs[0]
    
#Cells
locations = "/data/photos/999.png"
loc = locations.replace("/data/photos/", "").replace(".png", "")

print(locations)
print(loc)